# Application

In [20]:
import sys
import traitlets
from traitlets.config import Application
Application._instance = None

# During the start of the notebook, different CLI parameters can be passed to
# the process that literally keeps the notebook. Traitlets are implemented in
# such a way that they take arguments that are common to the environment. The
# following code removes all additional arguments so that the outputs are not
# affected by arguments specified for the jupyter run.
sys.argv = [arg for arg in sys.argv if not arg.startswith("--")]

## Start

The method `start` is called when the application starts. It is usually used as an action code for the application, most commonly to start the application loop.

---

The following cell defines the ancestor of the `Application`, which overloads the `start` method to `print("application started")`.

In [2]:
from traitlets import Unicode
class CustomApplication(Application):
    f = Unicode("some").tag(config=True)
    def start(self):
        print("application started")

When the application launches, it prints the corresponding message to the standard output.

In [15]:
application = CustomApplication()
application.launch_instance()

application started


## Command line

The core feature of the `Application` class is that it behaves like `traitlets.config.Config`, but it can override class properties from the command line arguments. You just have to pass arguments like `<class name>.<attribute>=<value>` when calling the program, just in the configuration script for the `traitlets.config.Config`.

---

The following cell defines python script that defines the inheritor of the `Application`, that has the `parameter` trait in it - consider how to set the value of `parameter`.

In [ ]:
%%writefile /tmp/my_application.py
import traitlets
from traitlets.config import Application
from traitlets.utils import cast_unicode

class MyApplication(Application):
    parameter = traitlets.Unicode(
        default_value="default",
        config=True
    )

if __name__ == "__main__":
    MyApplication.launch_instance()
    my_application = MyApplication.instance()
    print(my_application.parameter)

Writing /tmp/my_application.py


The following cell calls the previously created scriptip without any additional arguments.

In [10]:
!python3 /tmp/my_application.py

default


As a result, the `MyApplication.instance().parameter` will have a default value. The following cell runs the same script, but passes the value as a CLI argument.

In [3]:
!python3 /tmp/my_application.py --MyApplication.parameter="new value"

new value


Sometimes it usefull to intialise `Application` from python code. You can do this by passing argumets as a `list[str]` to the `initialize` method.

In [11]:
import os
os.chdir("/tmp")
from my_application import MyApplication

app = MyApplication()
app.initialize(["--MyApplication.parameter", "new value"])
app.parameter

'new value'

### Not recognized options

If the `Application` detects arguments during launch that weren't specified in the application class, it'll throw corresponding messages to the output.

---

The following cell defines the child of the `Application` with **only** `correct_arg` in it.

In [21]:
class ExampleApplication(Application):
    correct_arg = traitlets.Unicode("default").tag(config=True)

The next code launches the application on the class defined earlier, but passes two arguments, one of which was no specified as a trait.

In [22]:
ExampleApplication._instance = None
ExampleApplication.launch_instance(
    argv=[
        "--ExampleApplication.wrong_arg", "original",
        "--ExampleApplication.correct_arg", "new"
    ]
)
ExampleApplication.instance().correct_arg

[ExampleApplication] WARNING | Config option `wrong_arg` not recognized by `ExampleApplication`.


'new'

As a result, there is the message indicating that the application doesn't recognize the argument.

The most annoying consequence is that if the program is runned with arguments that are not supposed to be used by this application, you will receive a corresponding message.

---

The following cell adds some arguments to the `sys.argv` to simulate them being passed during interpreter startup.

In [23]:
sys.argv.append("--wrong_arg_from_sys=new value")
sys.argv.append("--ExampleApplication.correct_arg=new value")

Launching the application is accompained by the described warnings.

In [24]:
ExampleApplication.launch_instance()
ExampleApplication().instance().correct_arg

[ExampleApplication] WARNING | Unrecognized alias: 'wrong_arg_from_sys', it will have no effect.


'new value'